<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/ml801/dataset-filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!apt install git-lfs
from huggingface_hub import notebook_login
notebook_login()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.


## Loading Dataset

In [ ]:
!pip install datasets

In [25]:
from datasets import load_dataset

dataset = load_dataset("HamdanXI/arb-eng-parallel-10k-splitted")

## Cosine Similarity / Euclidean Distance

In [3]:
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.1 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=e84e196dac1bd99fb9c13fc32a60d137b5fb2e9519a331b5c43188d704b8fa65
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
from scipy.spatial import distance
from datasets import load_dataset
from tqdm import tqdm

# Load tokenizer and model (e.g., multilingual BERT)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model = AutoModel.from_pretrained("bert-base-multilingual-cased")

def embed_sentence(sentence, lang):
    tokens = tokenizer(sentence, return_tensors='pt', max_length=128, truncation=True, padding='max_length')
    with torch.no_grad():
        embeddings = model(**tokens)
    return embeddings.last_hidden_state.mean(dim=1)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [ ]:
# Initialize variables to store the maximum lengths
max_length_arabic = 0
max_length_english = 0

# Iterate over the dataset to find the maximum lengths
for row in dataset['train']:
    length_arabic = len(row['arabic'].split())  # Number of words in the Arabic sentence
    length_english = len(row['english'].split())  # Number of words in the English sentence

    # Update the maximum lengths if current lengths are greater
    if length_arabic > max_length_arabic:
        max_length_arabic = length_arabic
    if length_english > max_length_english:
        max_length_english = length_english

if max_length_arabic > max_length_english:
  max_length = max_length_arabic
else:
  max_length = max_length_english

print(max_length)

449


In [24]:
import numpy as np

def calculate_cosine_similarity(embedding1, embedding2):
    return cosine_similarity(embedding1, embedding2)[0][0]

def calculate_euclidean_distance(embedding1, embedding2):
    # Flatten embeddings to ensure they are 1-D
    embedding1 = np.array(embedding1).flatten()
    embedding2 = np.array(embedding2).flatten()
    return distance.euclidean(embedding1, embedding2)

In [12]:
# Initialize an empty list to store filtered data
filtered_data_cosine = []

# Define a similarity threshold
similarity_threshold = 0.6  # Adjust this based on your needs

for row in tqdm(dataset['train'], desc="Processing", unit="row"):
    arabic_embedding = embed_sentence(row['arabic'], 'ar')
    english_embedding = embed_sentence(row['english'], 'en')

    similarity = calculate_cosine_similarity(arabic_embedding, english_embedding)
    if similarity > similarity_threshold:
        filtered_data_cosine.append(row)

Processing: 100%|██████████| 8000/8000 [14:40<00:00,  9.08row/s]


In [13]:
from datasets import Dataset

# Assuming filtered_data is a list of dictionaries
# Convert the filtered data to a dictionary format
data_dict = {"arabic": [row['arabic'] for row in filtered_data_cosine],
             "english": [row['english'] for row in filtered_data_cosine]}

# Create a new dataset from the dictionary
filtered_dataset_6 = Dataset.from_dict(data_dict)

# Now filtered_dataset is a Hugging Face Dataset containing your filtered data

In [8]:
filtered_dataset

Dataset({
    features: ['arabic', 'english'],
    num_rows: 7950
})

In [11]:
filtered_dataset_5

Dataset({
    features: ['arabic', 'english'],
    num_rows: 6489
})

In [14]:
filtered_dataset_6

Dataset({
    features: ['arabic', 'english'],
    num_rows: 2771
})

In [20]:
dataset['train'] = filtered_dataset_5

In [21]:
dataset.push_to_hub("arb-eng-parallel-10k-splitted-cosine-5")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# Initialize an empty list to store filtered data
filtered_data_euclidean = []

# Define a similarity threshold
distance_threshold = 0.5  # Adjust this based on your needs

for row in tqdm(dataset['train'], desc="Processing", unit="row"):
    arabic_embedding = embed_sentence(row['arabic'], 'ar')
    english_embedding = embed_sentence(row['english'], 'en')

    euclidean_dist = calculate_euclidean_distance(arabic_embedding, english_embedding)
    if euclidean_dist < distance_threshold:
        filtered_data_euclidean.append(row)

Processing:   2%|▏         | 169/8000 [00:28<15:06,  8.64row/s]

In [29]:
print(arabic_embedding.shape)

torch.Size([1, 768])
